In [1]:
import os
import requests
import json

In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')


def upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1):
  
    url = "https://content-api.wildberries.ru/content/v3/media/file"
    
    headers = {
        "Authorization": api_key,
        "X-Nm-Id": str(nm_id),
        "X-Photo-Number": str(photo_number)
    }
    
    with open(file_path, "rb") as f:
        files = {
            "uploadfile": (os.path.basename(file_path), f, "image/jpeg")
        }
        
        try:
            response = requests.post(url, headers=headers, files=files)
        except requests.RequestException as e:
            print(f"Ошибка при отправке запроса: {e}")
            return
        
        if response.status_code == 200:
            resp_json = response.json()
            if not resp_json.get("error", False):
                print(f"Успешно загружено изображение '{os.path.basename(file_path)}' на nmID={nm_id}.")
            else:
                print(f"Ошибка Wildberries при загрузке: {resp_json.get('errorText', 'Неизвестная ошибка')}")
        else:
            print(f"Ошибка ответа сервера. Код: {response.status_code}, файл: {os.path.basename(file_path)}")

def process_vendor_list(vendor_list, images_folder, api_key):
    """
    Проходит по списку vendor_list, проверяет, существует ли соответствующий файл в images_folder,
    и при наличии вызывает функцию загрузки на WB.
    
    :param vendor_list: Список словарей вида [{'nmID': ..., 'vendorCode': 'box_XXXX_YY'}, ...]
    :param images_folder: Путь к папке, где лежат файлы (например, 'images_900_1200').
    :param api_key: API-ключ для Wildberries.
    """
    for item in vendor_list:
        nm_id = item["nmID"]
        vendor_code = item["vendorCode"]
        
        # Ожидаем формат vendorCode = "box_9604677_30"
        # Удаляем "box_" в начале:
        if vendor_code.startswith("box_"):
            code_part = vendor_code[4:]  # "9604677_30"
            # Разделяем по символу "_"
            parts = code_part.split("_", maxsplit=1)
            if len(parts) == 2:
                product_id, quantity_str = parts
                # Формируем имя файла: "9604677-30.jpg"
                image_name = f"{product_id}-{quantity_str}.jpg"
                file_path = os.path.join(images_folder, image_name)
                
                # Проверяем, существует ли файл
                if os.path.isfile(file_path):
                    print(f"Файл '{image_name}' найден. Загружаем на nmID={nm_id}...")
                    upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1)
                else:
                    print(f"Файл '{image_name}' не найден в папке '{images_folder}'. Пропускаем.")
            else:
                print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")
        else:
            print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")


In [23]:
all_cards_list = "https://content-api.wildberries.ru/content/v2/get/cards/list"
headers = {
    "Authorization": WB_TOKEN,
}
body =             {
          "settings": {                      
            "cursor": {
              "limit": 100
            },
            "filter": {
              "withPhoto": 0
            }
          }
        }


resp = requests.post(all_cards_list, headers=headers, json=body)

In [24]:
d = json.loads(resp.text)
d

{'cards': [{'nmID': 322405292,
   'imtID': 303872301,
   'nmUUID': '0194b77c-05f9-7efe-a534-f03de84b8eab',
   'subjectID': 3979,
   'subjectName': 'Коробки картонные',
   'vendorCode': 'box_9602797_50',
   'brand': '',
   'title': 'Короб КС-404 МГФ, белый, 100*80*30 мм 50 шт.',
   'needKiz': False,
   'dimensions': {'width': 36, 'height': 15, 'length': 26, 'isValid': True},
   'characteristics': [{'id': 15002205,
     'name': 'Марка картона',
     'value': ['Т-23']},
    {'id': 14177449, 'name': 'Цвет', 'value': ['крафт']},
    {'id': 15000432, 'name': 'Форма коробки', 'value': ['Плоская']},
    {'id': 90630, 'name': 'Высота предмета', 'value': 3},
    {'id': 14177451, 'name': 'Страна производства', 'value': ['Россия']},
    {'id': 88952, 'name': 'Вес товара с упаковкой (г)', 'value': 1000},
    {'id': 90673, 'name': 'Ширина предмета', 'value': 8},
    {'id': 179792, 'name': 'Количество предметов в упаковке', 'value': ['50']},
    {'id': 90652, 'name': 'Глубина предмета', 'value': 10}]

In [25]:
newNmIdVendorCodes = [{"nmID": row["nmID"], "vendorCode": row["vendorCode"]} for row in d["cards"]]

In [26]:
newNmIdVendorCodes

[{'nmID': 322405292, 'vendorCode': 'box_9602797_50'},
 {'nmID': 322405274, 'vendorCode': 'box_9604368_10'},
 {'nmID': 322405255, 'vendorCode': 'box_9602818_30'},
 {'nmID': 322405219, 'vendorCode': 'box_9602829_10'},
 {'nmID': 322405175, 'vendorCode': 'box_9604625_50'},
 {'nmID': 322405166, 'vendorCode': 'box_9601611_50'},
 {'nmID': 322405164, 'vendorCode': 'box_9602830_50'},
 {'nmID': 322405161, 'vendorCode': 'box_9602826_50'},
 {'nmID': 322405156, 'vendorCode': 'box_9602807_30'},
 {'nmID': 322405153, 'vendorCode': 'box_9603049_10'},
 {'nmID': 322405149, 'vendorCode': 'box_222015_10'},
 {'nmID': 322405142, 'vendorCode': 'box_222193_50'}]

In [27]:
process_vendor_list(newNmIdVendorCodes, "./images_900_1200", WB_TOKEN)

Файл '9602797-50.jpg' найден. Загружаем на nmID=322405292...
Успешно загружено изображение '9602797-50.jpg' на nmID=322405292.
Файл '9604368-10.jpg' найден. Загружаем на nmID=322405274...
Успешно загружено изображение '9604368-10.jpg' на nmID=322405274.
Файл '9602818-30.jpg' найден. Загружаем на nmID=322405255...
Успешно загружено изображение '9602818-30.jpg' на nmID=322405255.
Файл '9602829-10.jpg' найден. Загружаем на nmID=322405219...
Успешно загружено изображение '9602829-10.jpg' на nmID=322405219.
Файл '9604625-50.jpg' найден. Загружаем на nmID=322405175...
Успешно загружено изображение '9604625-50.jpg' на nmID=322405175.
Файл '9601611-50.jpg' найден. Загружаем на nmID=322405166...
Успешно загружено изображение '9601611-50.jpg' на nmID=322405166.
Файл '9602830-50.jpg' найден. Загружаем на nmID=322405164...
Успешно загружено изображение '9602830-50.jpg' на nmID=322405164.
Файл '9602826-50.jpg' найден. Загружаем на nmID=322405161...
Успешно загружено изображение '9602826-50.jpg' на 